In [1]:
import pandas as pd
import os

# # Directory where the CSV files are stored
# input_dir = 'data_dyad_monthly'
# 
# # List all CSV files in the directory
# csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]
# csv_files = list(sorted(csv_files))
# 
# # Initialize an empty list to store DataFrames
# dfs = []
# 
# # Loop over the list of CSV files
# for file in sorted(csv_files):  # Sorting ensures the files are processed in order
#     print(f'Reading {file}')
#     file_path = os.path.join(input_dir, file)
#     # Read the current CSV into a DataFrame
#     df = pd.read_csv(file_path)
#     # Append the DataFrame to the list
#     dfs.append(df)

# Concatenate all DataFrames in the list into one
dyad_df = pd.read_parquet('data_dyad_monthly/dyad_df.parquet')
dyad_df

,month_id,date,country_id_a,country_id_b,a_country_name,b_country_name,a_gleditsch_ward,a_ged_sb,a_ged_ns,a_ged_os,...,b_renewable_internal_pcap_t48,b_renewable_pcap_t48,b_splag_1_decay_ged_sb_5,b_splag_1_decay_ged_os_5,b_splag_1_decay_ged_ns_5,ratio_wdi_sp_pop_totl,ratio_ged_sb,ratio_wdi_sp_dyn_imrt_in,ratio_vdem_v2x_ex_military,ratio_wdi_ms_mil_xpnd_zs
0,121,1990-01-01,1,2,Guyana,Suriname,110,0.0,0.0,0.0,...,264606.831667,264606.831667,0.062500,0.062500,0.062500,1.843709,0.0,1.197500,2.500000,3.675305
1,121,1990-01-01,1,3,Guyana,Trinidad and Tobago,110,0.0,0.0,0.0,...,3244.939711,3244.939711,0.000000,0.000000,0.000000,0.599340,0.0,1.761029,0.250000,2.032451
2,121,1990-01-01,1,4,Guyana,Venezuela,110,0.0,0.0,0.0,...,45295.968642,74555.476336,1.062500,1.062500,0.093750,0.039060,0.0,1.863813,1.748252,3.675305
3,121,1990-01-01,1,5,Guyana,Samoa,110,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.484573,0.0,1.871094,0.250000,3.675305
4,121,1990-01-01,1,6,Guyana,Tonga,110,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,7.653026,0.0,2.443878,0.250000,3.675305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6687689,502,2021-10-01,243,245,Morocco,Sudan,600,0.0,0.0,0.0,...,98.007034,926.166474,5.816131,4.897757,4.263890,0.825571,0.0,0.403509,0.800000,1.286171
6687690,502,2021-10-01,243,246,Morocco,South Sudan,600,0.0,0.0,0.0,...,2382.968084,4536.804621,4.950636,4.593437,4.628575,3.459173,0.0,0.252351,0.250000,1.965578
6687691,502,2021-10-01,244,245,Mauritania,Sudan,435,0.0,0.0,0.0,...,98.007034,926.166474,5.816131,4.897757,4.263890,0.101228,0.0,0.827068,2.800000,1.442215
6687692,502,2021-10-01,244,246,Mauritania,South Sudan,435,0.0,0.0,0.0,...,2382.968084,4536.804621,4.950636,4.593437,4.628575,0.424147,0.0,0.517241,0.875000,2.204050


In [ ]:
save_to_parquet = False
if save_to_parquet:
    dyad_df.to_parquet('data_dyad_monthly/dyad_df.parquet')
# save_to_csv = True
# if save_to_csv:
#     dyad_df.to_csv('data_dyad_monthly/dyad_df.csv', index=True)

In [ ]:
dyad_df.info()

In [ ]:
# explore data
# plot distribution of a_ged_sb
import matplotlib.pyplot as plt

dyad_df['a_ged_sb'].plot.hist(bins=20)
# log scale
plt.yscale('log')
plt.title('Distribution of a_ged_sb')
plt.show()
dyad_df['b_ged_sb'].plot.hist(bins=20)
plt.yscale('log')
plt.title('Distribution of b_ged_sb')
plt.show()
# plot number of dyads where according to count of a_ged_sb and b_ged_sb
# 0 if both are 0, 1 if one is 0, 2 if both are not 0


In [ ]:

# Function to categorize each dyad
def categorize_dyads(a, b):
    if a == 0 and b == 0:
        return 0
    elif a == 0 or b == 0:
        return 1
    else:
        return 2


# Apply the function to each row and create a new column with the category
dyad_df['category'] = dyad_df.apply(lambda x: categorize_dyads(x['a_ged_sb'], x['b_ged_sb']), axis=1)

# Count occurrences of each category
category_counts = dyad_df['category'].value_counts().sort_index()

# Plot
category_counts.plot(kind='bar')
plt.title('Number of Dyads by Category')
plt.xlabel('Category')
plt.ylabel('Number of Dyads')
plt.xticks(ticks=[0, 1, 2], labels=['Both 0', 'One 0', 'Both not 0'], rotation=0)
plt.show()

In [ ]:
dyad_df.sort_values(by=['country_id_a', 'country_id_b', 'month_id'], inplace=True)


# Function to shift the 'ged_sb' variable backwards by 15 months within each group
def shift_ged_sb(group):
    # Assuming 'ged_sb' is recorded per country in the dyad, adjust if you have a single 'ged_sb' variable
    group['a_ged_sb_shifted'] = group['a_ged_sb'].shift(-15)
    group['b_ged_sb_shifted'] = group['b_ged_sb'].shift(-15)
    return group


# Apply the function to each dyad group
dyad_df = dyad_df.groupby(['country_id_a', 'country_id_b']).apply(shift_ged_sb)
dyad_df.reset_index(drop=True, inplace=True)
# cm_features['y_shifted'] = cm_features.groupby('country_id')['ged_sb'].shift(-15)  # 3 months gap + 12 months prediction
# show na for y_shifted
# cm_features[cm_features['y_shifted'].isna()]
# drop na
dyad_df = dyad_df.dropna()
dyad_df

In [ ]:
# transform from dyad to country/month
# per each month, aggregate the dyad data by country and then average fatalities
# based on first country only as we have duplicates

# Step 1: Split the DataFrame into two, one for each country in the dyad
df_a = dyad_df[['month_id', 'country_id_a', 'a_country_name', 'a_ged_sb', 'a_ged_sb_shifted']].copy()
df_b = dyad_df[['month_id', 'country_id_b', 'b_country_name', 'b_ged_sb', 'b_ged_sb_shifted']].copy()

# Step 2: Rename columns to have a consistent naming convention
df_a.rename(columns={'country_id_a': 'country_id', 'a_country_name': 'country_name', 'a_ged_sb': 'ged_sb',
                     'a_ged_sb_shifted': 'y_shifted'},
            inplace=True)
df_b.rename(columns={'country_id_b': 'country_id', 'b_country_name': 'country_name', 'b_ged_sb': 'ged_sb',
                     'b_ged_sb_shifted': 'y_shifted'},
            inplace=True)

# Step 3: Concatenate these DataFrames into a single DataFrame
combined_country_df = pd.concat([df_a, df_b], ignore_index=True)

# Step 4: Group by month_id and country_id, then aggregate to average the fatalities
# Adjusting for the possibility of no fatalities data, replace NaN with 0 or a suitable default before averaging if necessary
cm_features = combined_country_df.groupby(['month_id', 'country_id', 'country_name']).agg(
    {'ged_sb': 'mean', 'y_shifted': 'mean'}).reset_index()

# aggregated_df now contains the average fatalities per country per month
cm_features

In [ ]:

# plot y_shifted per month
from matplotlib import pyplot as plt

cm_features['y_shifted'].plot()
cm_features['ged_sb'].plot()
# cm_features['ged_sb_tlag_6'].plot()
plt.legend()
plt.show()